In [5]:
import pandas as pd
import nltk
from sklearn.utils import shuffle

# Loading our datafile as a DataFrame with pandas

df = pd.read_csv('/Users/laniepreston/TMCI_Edits/group_assignment/FullReviews.csv' , usecols=[6,9], header=0)
shuf_red_df = shuffle(df[:10000])
print(shuf_red_df)

      Score                                               Text
2123      4  This freeze dried ice cream is great! It taste...
5779      5  I've ordered these before and I am still thril...
7331      5  I HAVE SO MUCH FUN WATCHING MY PUPPIES ASKING ...
2383      5  I am a coffee snob. Yes, I admit it. I prefer ...
9626      5  I like the smoked sweet paprika better than ot...
6193      5  I LOVE these chips.  They are addictive...it's...
7842      1  Wolfgang Puck recently changed their Chicken s...
4858      5  Newman's Own Food is one of the best on the ma...
6116      5  My cat suffers from bladder cysts frequently s...
4211      5  This product is great for pancakes. For baked ...
8798      5  I bought this for my son who travels often - h...
5855      5  I ordered a set of three cans and they showed ...
7225      5  The grapefruit and blackberry flavors are the ...
4075      1  This product was awesome the first couple mont...
2010      5  I bought a Belgian waffle maker sort of on

In [16]:
# PoS-tagging our dataset
train_df = shuf_red_df[:8000]
test_df = shuf_red_df[:2000]

train_tagged = []
test_tagged = []

for index, item in train_df.iterrows():
    score = item[0]
    review = item[1]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    train_tagged.append([rev, score])

for index, item in test_df.iterrows():
    score = item[0]
    review = item[1]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    test_tagged.append([rev, score])

In [17]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
# Lemmatizing our dataset + filtering for stopwords (except negation)
training_dicts = dict((k,[]) for k in [1,2,3,4,5])
test_dicts = dict((k,[]) for k in [1,2,3,4,5])

stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

lemmatized = []

for review in train_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
            else: 
                continue
        
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    training_dicts[score].append(lem_rev)

In [18]:
for review in test_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    test_dicts[score].append(lem_rev)

In [19]:
import pickle
# Save lemmatized text to a pickle file
pickle.dump(training_dicts, open("training_dicts.txt", "wb"))
pickle.dump(test_dicts, open("test_dicts.txt", "wb"))
